In [1]:
import numpy as np
import librosa.display, os
import matplotlib.pyplot as plt
%matplotlib inline
import matplotlib.pyplot as plt
from keras.preprocessing import image
import os
import numpy as np
import tensorflow as tf
import numpy as np
import librosa.display, os
import matplotlib.pyplot as plt
%matplotlib inline
import matplotlib.pyplot as plt 
from keras.preprocessing import image
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, Flatten, Dense, concatenate, Dropout
from tensorflow.keras.optimizers import Adam
import os
import soundfile as sf
import numpy as np
import librosa
from skimage.transform import resize
import pandas as pd
from tqdm import tqdm


mel_train = []
cqt_train = []
mel_test = []
cqt_test = []
label_train = []
label_test = []

2023-05-25 10:49:56.304697: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:98: UserWarning: unable to load libtensorflow_io_plugins.so: unable to open file: libtensorflow_io_plugins.so, from paths: ['/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so']
caused by: ['dlopen(/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so, 0x0006): symbol not found in flat namespace (__ZN3tsl6Status22MaybeAddSourceLocationENS_14SourceLocationE)']
  warnings.warn(f"unable to 

In [3]:
def load_audio_files(audio_folder):
    audio_files = []
    labels = []

    for root, dirs, files in os.walk(audio_folder):
        for file in files:
            if file.endswith(".wav"):
                audio_files.append(os.path.join(root, file))
                labels.append(os.path.basename(root))

    return audio_files, labels
def process_audio(file_path):
    sample_rate = 32000
    n_mfcc = 13
    n_fft = 2048
    hop_length = 512

    # Load audio file
    audio, sample_rate = librosa.load(file_path, sr=sample_rate)

    # Compute Mel spectrogram
    mel = librosa.feature.melspectrogram(y=audio, sr=sample_rate, n_fft=n_fft, hop_length=hop_length, n_mels=128)
    # Compute MFCC from the Mel spectrogram
    mfcc = librosa.feature.mfcc(S=librosa.power_to_db(mel), n_mfcc=n_mfcc)

    # Resize spectrograms
    mfcc_resized = resize(mfcc, (224, 224, 3), mode='reflect', anti_aliasing=True)
    mel_resized = resize(mel, (224, 224, 3), mode='reflect', anti_aliasing=True)

    # Clear memory
    del audio

    return mfcc_resized, mel_resized
def process_audio_files(audio_files, audio_folder):
    mfcc_train = []
    mel_train = []
    y_train = []

    total_files = len(audio_files)

    with tqdm(total=total_files, ncols=80) as pbar:
        for file_path in audio_files:
            if file_path.endswith(".wav"):
                # Process audio and extract spectrograms
                mfcc, mel = process_audio(file_path)
                # Append spectrograms and label to the train arrays
                mfcc_train.append(mfcc)
                mel_train.append(mel)
                # Extract label from subfolder name
                label = os.path.basename(os.path.dirname(file_path))
                y_train.append(label)
                pbar.set_postfix({'file': file_path})
                pbar.update()

    # Convert train arrays to NumPy arrays
    mfcc_train = np.array(mfcc_train)
    mel_train = np.array(mel_train)
    y_train = np.array(y_train)

    return mfcc_train, mel_train, y_train
def main(audio_folder):
    audio_files, labels = load_audio_files(audio_folder)
    X_mfcc, X_mel, y = process_audio_files(audio_files, audio_folder)
    return X_mfcc, X_mel, y


In [5]:
# Usage
audio_folder = ''                        
mfcc_test, mel_test, label_test = main(audio_folder)

100%|█| 152/152 [00:09<00:00, 15.64it/s, file=/Users/sammerking/Desktop/Datasets


In [6]:
# Usage
audio_folder = ''                        
mfcc_train, mel_train, label_train = main(audio_folder)

100%|█| 835/835 [00:56<00:00, 14.84it/s, file=/Users/sammerking/Desktop/Datasets


In [7]:
from sklearn.preprocessing import LabelEncoder

# Create an instance of LabelEncoder
label_encoder = LabelEncoder()

# Fit the encoder on your label data
label_encoder.fit(label_train)

# Transform the labels to numerical values
y_train_encoded = label_encoder.transform(label_train)
y_test_encoded = label_encoder.transform(label_test)

# Get the unique class names
class_names = label_encoder.classes_

# Create a dictionary to map numerical labels to class names
label_to_class = {i: class_name for i, class_name in enumerate(class_names)}

# Convert the encoded labels to one-hot encoded vectors
label_train_encoded = to_categorical(y_train_encoded)
label_test_encoded = to_categorical(y_test_encoded)


In [8]:
from tensorflow.keras.utils import to_categorical

mel_train_norm = np.array(mel_train) / 255  
mel_test_norm = np.array(mel_test) / 255      
mfcc_train_norm = np.array(mfcc_train) / 255    
mfcc_test_norm = np.array(mfcc_test) / 255    

In [9]:
# Define input layers
mel_input = Input(shape=(224, 224, 3))  # Modify the shape according to your mel spectrogram dimensions
cqt_input = Input(shape=(224, 224, 3))

# Convolutional layers for Mel spectrograms
mel_conv = Conv2D(32, kernel_size=(3, 3), activation='relu')(mel_input)
mel_conv = MaxPooling2D(pool_size=(2, 2))(mel_conv)
mel_conv = Conv2D(64, kernel_size=(3, 3), activation='relu')(mel_conv)
mel_conv = MaxPooling2D(pool_size=(2, 2))(mel_conv)
mel_conv = Conv2D(128, kernel_size=(3, 3), activation='relu')(mel_conv)
mel_conv = MaxPooling2D(pool_size=(2, 2))(mel_conv)
mel_conv = Flatten()(mel_conv)

# Convolutional layers for CQT spectrograms
cqt_conv = Conv2D(32, kernel_size=(3, 3), activation='relu')(cqt_input)
cqt_conv = MaxPooling2D(pool_size=(2, 2))(cqt_conv)
cqt_conv = Conv2D(64, kernel_size=(3, 3), activation='relu')(cqt_conv)
cqt_conv = MaxPooling2D(pool_size=(2, 2))(cqt_conv)
cqt_conv = Conv2D(128, kernel_size=(3, 3), activation='relu')(cqt_conv)
cqt_conv = MaxPooling2D(pool_size=(2, 2))(cqt_conv)
cqt_conv = Flatten()(cqt_conv)

# Merge the branches
merged = concatenate([mel_conv, cqt_conv])

# Dense layers for classification
dense = Dense(256, activation='relu')(merged)
dense = Dropout(0.5)(dense)  # Add dropout with a rate of 0.5
dense = Dense(128, activation='relu')(dense)
dense = Dropout(0.5)(dense)  # Add dropout with a rate of 0.5
output = Dense(5, activation='softmax')(dense)

# Define the model
model = Model(inputs=[mel_input, cqt_input], outputs=output)

# Compile the model
model.compile(loss='categorical_crossentropy', optimizer=Adam(), metrics=['accuracy'])


In [10]:
from tensorflow.keras.callbacks import EarlyStopping

# Define early stopping callback
earlystop_callback = EarlyStopping(
    monitor='val_accuracy', patience=5, mode='max', verbose=1, restore_best_weights=True
)



In [11]:
hist = model.fit([mel_train_norm, mfcc_train_norm], label_train_encoded,
           batch_size=32, epochs=100, 
          validation_data=([mel_test_norm, mfcc_test_norm], label_test_encoded),
          callbacks=[earlystop_callback]
          )

Epoch 1/100
27/27 [==============================] - 58s 2s/step - loss: 1.6010 - accuracy: 0.3760 - val_loss: 1.0849 - val_accuracy: 0.6776
Epoch 2/100
27/27 [==============================] - 49s 2s/step - loss: 1.3297 - accuracy: 0.4659 - val_loss: 0.8444 - val_accuracy: 0.7697
Epoch 3/100
27/27 [==============================] - 50s 2s/step - loss: 1.1795 - accuracy: 0.5449 - val_loss: 0.8194 - val_accuracy: 0.7829
Epoch 4/100
27/27 [==============================] - 51s 2s/step - loss: 1.0712 - accuracy: 0.5749 - val_loss: 0.7393 - val_accuracy: 0.7763
Epoch 5/100
27/27 [==============================] - 48s 2s/step - loss: 0.8542 - accuracy: 0.6635 - val_loss: 0.8638 - val_accuracy: 0.8158
Epoch 6/100
27/27 [==============================] - 48s 2s/step - loss: 0.7934 - accuracy: 0.6922 - val_loss: 0.7378 - val_accuracy: 0.7895
Epoch 7/100
27/27 [==============================] - 48s 2s/step - loss: 0.6536 - accuracy: 0.7461 - val_loss: 0.7191 - val_accuracy: 0.8289
Epoch 8/100
2

In [12]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

# Make predictions on the test data
y_pred = model.predict([mel_test_norm, mfcc_test_norm])

# Convert the predictions from one-hot encoded format to class labels
y_pred_labels = np.argmax(y_pred, axis=1)
y_true_labels = np.argmax(label_test_encoded, axis=1)

# Calculate evaluation metrics
accuracy = accuracy_score(y_true_labels, y_pred_labels)
precision = precision_score(y_true_labels, y_pred_labels, average='weighted')
recall = recall_score(y_true_labels, y_pred_labels, average='weighted')
f1 = f1_score(y_true_labels, y_pred_labels, average='weighted')


# Print the evaluation metrics
print("Accuracy:", accuracy)
print("Precision:", precision)
print("Recall:", recall)
print("F1 Score:", f1)




5/5 [==============================] - 2s 447ms/step
Accuracy: 0.8881578947368421
Precision: 0.8752732121000174
Recall: 0.8881578947368421
F1 Score: 0.8756137314795778


/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
